In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# RAG 기본 파이프라인(1-8단계)

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # 분할기
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [3]:
# 단계 1 : 문서 로드(Load Documents)
# 기본적으로 페이지 단위로 분할
loader = PyMuPDFLoader('data/SPRi AI 브리프 2023년12월호 F.pdf')
docs = loader.load()
print(len(docs))

23


In [4]:
# 단계 2: 문서 분할(Split Document)
text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_document = text_split.split_documents(docs)
len(split_document)

43

In [17]:
# 단계 3: 임베딩(Embedding) 생성
# embeddings = OpenAIEmbeddings()
model_name = "nlpai-lab/KoE5"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)
# embeddings = hf_embeddings.embed_documents(split_document)

In [9]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 저장
vectorstore = FAISS.from_documents(documents=split_document, embedding=hf_embeddings)

In [10]:
# 단계 5 : 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x37617f220>, search_kwargs={})

In [11]:
retriever.invoke("삼성전자가 개발한 생성AI가 이름은 무엇인가요?")

[Document(metadata={'source': 'data/SPRi AI 브리프 2023년12월호 F.pdf', 'file_path': 'data/SPRi AI 브리프 2023년12월호 F.pdf', 'page': 12, 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13462', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20231208132838+09'00'", 'modDate': "D:20231208132838+09'00'", 'trapped': ''}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 

In [12]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
You must in include 'page' number in your answer
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [13]:
# 단계 7 : 언어모델(LLM) 생성
# 모델(LLM) 생성
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

In [14]:
# 단계 8 : 체인(Chain) 생성 
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# 단계 9 : 체인 실행(Run Chain)
question = "구굴은 앤스로픽에 얼마를 투자했나요??"
response = chain.stream(question)
for token in response:
    print(token,end="",flush=True)

구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자했습니다. (페이지 13)